In [5]:
import requests
import os
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

# Download required NLTK resources
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

class MarketInsightsApp:
    def __init__(self, api_key="ecb16116faac468a8300016a53c68516"):
        self.api_key = api_key
        self.base_url = "https://newsapi.org/v2"
        self.stop_words = set(stopwords.words('english'))
        self.additional_stop_words = {"said", "says", "reported", "told", "according", "company", "market", "markets", "year"}
        self.stop_words.update(self.additional_stop_words)
    
    def fetch_news(self, query, days=7, language="en", sort_by="relevancy", page_size=25):
        """Fetch news articles related to the query"""
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # Format dates for API
        from_date = start_date.strftime("%Y-%m-%d")
        to_date = end_date.strftime("%Y-%m-%d")
        
        # Build URL
        url = f"{self.base_url}/everything"
        
        # Parameters
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": language,
            "sortBy": sort_by,
            "pageSize": page_size,
            "apiKey": self.api_key
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise exception for 4XX/5XX errors
            data = response.json()
            
            if data.get("status") == "ok":
                return data
            else:
                print(f"API Error: {data.get('message', 'Unknown error')}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            return None
    
    def generate_summary(self, articles, max_sentences=5):
        """Generate a summary from a list of articles"""
        if not articles:
            return "No articles found to summarize."
        
        # Combine all article content
        text_parts = []
        for a in articles:
            desc = a.get("description", "") or ""  # Convert None to empty string
            content = a.get("content", "") or ""  # Convert None to empty string
            if desc or content:  # Only add if we have some content
                text_parts.append(desc + " " + content)
        
        all_text = " ".join(text_parts)
        
        # If we don't have enough text to summarize
        if not all_text.strip():
            return "Insufficient content for summarization."
        
        # Clean text
        all_text = re.sub(r'\[.*?\]', '', all_text)  # Remove content in brackets
        all_text = re.sub(r'http\S+', '', all_text)  # Remove URLs
        
        # Tokenize sentences
        sentences = sent_tokenize(all_text)
        
        # Calculate word frequencies
        words = word_tokenize(all_text.lower())
        words = [word for word in words if word.isalnum() and word not in self.stop_words]
        word_freq = FreqDist(words)
        
        # Score sentences based on word frequency
        sentence_scores = {}
        for sentence in sentences:
            for word in word_tokenize(sentence.lower()):
                if word in word_freq:
                    if sentence not in sentence_scores:
                        sentence_scores[sentence] = 0
                    sentence_scores[sentence] += word_freq[word]
        
        # Get top sentences
        if not sentence_scores:
            return "Could not generate a meaningful summary from the available content."
            
        summary_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:max_sentences]
        summary = " ".join([s[0] for s in summary_sentences])
        
        return summary
    
    def analyze_sentiment(self, articles):
        """Analyze sentiment of articles"""
        if not articles:
            return {"overall": "neutral", "polarity": 0, "positive": 0, "neutral": 1, "negative": 0}
        
        # Safely join text
        text_parts = []
        for a in articles:
            title = a.get("title", "") or ""
            desc = a.get("description", "") or ""
            if title or desc:
                text_parts.append(title + " " + desc)
        
        all_text = " ".join(text_parts)
        
        # Use TextBlob for sentiment analysis
        blob = TextBlob(all_text)
        polarity = blob.sentiment.polarity
        
        # Map polarity to categories
        if polarity > 0.1:
            sentiment = "positive"
        elif polarity < -0.1:
            sentiment = "negative"
        else:
            sentiment = "neutral"
        
        # Calculate sentiment percentages
        sentiments = []
        for a in articles:
            title = a.get("title", "") or ""
            desc = a.get("description", "") or ""
            if title or desc:
                sentiments.append(TextBlob(title + " " + desc).sentiment.polarity)
        
        pos = sum(1 for s in sentiments if s > 0.1) / len(sentiments) if sentiments else 0
        neg = sum(1 for s in sentiments if s < -0.1) / len(sentiments) if sentiments else 0
        neu = sum(1 for s in sentiments if -0.1 <= s <= 0.1) / len(sentiments) if sentiments else 1
        
        return {
            "overall": sentiment,
            "polarity": polarity,
            "positive": pos, 
            "neutral": neu, 
            "negative": neg
        }
    
    def extract_key_topics(self, articles, n=5):
        """Extract key topics from articles"""
        if not articles:
            return []
        
        # Safely combine all titles and descriptions
        text_parts = []
        for a in articles:
            title = a.get("title", "") or ""
            desc = a.get("description", "") or ""
            if title or desc:
                text_parts.append(title + " " + desc)
        
        all_text = " ".join(text_parts)
        
        # Tokenize and remove stop words
        words = word_tokenize(all_text.lower())
        words = [word for word in words if word.isalnum() and word not in self.stop_words and len(word) > 2]
        
        # Get most common words
        word_counts = Counter(words)
        return word_counts.most_common(n)
    
    def extract_companies(self, articles):
        """Extract mentioned companies from articles (basic implementation)"""
        # This is a simplified implementation
        # For production, use Named Entity Recognition (NER)
        if not articles:
            return []
        
        # Safely combine all titles and descriptions
        text_parts = []
        for a in articles:
            title = a.get("title", "") or ""
            desc = a.get("description", "") or ""
            if title or desc:
                text_parts.append(title + " " + desc)
        
        all_text = " ".join(text_parts)
        
        # Simple regex to find potential company names (very basic)
        companies = re.findall(r'[A-Z][a-z]+ (?:Inc|Corp|Ltd|LLC|Co|Company|Corporation)', all_text)
        companies.extend(re.findall(r'[A-Z][A-Z]+', all_text))  # Stock symbols
        
        # Count occurrences
        company_counts = Counter(companies)
        return company_counts.most_common(10)
    
    def get_market_insights(self, query):
        """Generate comprehensive market insights for a query"""
        # Fetch news
        data = self.fetch_news(query)
        if not data or not data.get("articles"):
            return {
                "status": "error",
                "message": f"No articles found for '{query}'. Try a different search term."
            }
        
        articles = data.get("articles", [])
        
        # Generate insights
        summary = self.generate_summary(articles)
        sentiment = self.analyze_sentiment(articles)
        topics = self.extract_key_topics(articles)
        companies = self.extract_companies(articles)
        
        # Add source diversity
        sources = [a.get("source", {}).get("name") for a in articles if a.get("source")]
        source_counts = Counter([s for s in sources if s])  # Filter out None values
        
        # Format the results
        insights = {
            "status": "success",
            "query": query,
            "article_count": len(articles),
            "summary": summary,
            "sentiment": sentiment,
            "key_topics": topics,
            "mentioned_companies": companies,
            "sources": source_counts.most_common(5),
            "articles": [
                {
                    "title": a.get("title"),
                    "source": a.get("source", {}).get("name"),
                    "published_at": a.get("publishedAt"),
                    "url": a.get("url")
                } for a in articles[:5] if a.get("title")  # Top 5 articles
            ]
        }
        
        return insights
    
    def print_insights(self, insights):
        """Print insights in a readable format"""
        if insights.get("status") == "error":
            print(f"Error: {insights.get('message')}")
            return
        
        print("\n" + "=" * 80)
        print(f"MARKET INSIGHTS: {insights['query'].upper()}")
        print("=" * 80)
        
        print("\n📊 OVERVIEW:")
        print(f"• Found {insights['article_count']} articles")
        print(f"• Overall market sentiment: {insights['sentiment']['overall'].upper()}")
        print(f"• Sentiment breakdown: {insights['sentiment']['positive']:.0%} positive, " 
              f"{insights['sentiment']['neutral']:.0%} neutral, {insights['sentiment']['negative']:.0%} negative")
        
        print("\n📝 MARKET SUMMARY:")
        print(insights['summary'])
        
        print("\n🔑 KEY TOPICS:")
        for topic, count in insights['key_topics']:
            print(f"• {topic.title()} ({count} mentions)")
        
        if insights['mentioned_companies']:
            print("\n🏢 MENTIONED COMPANIES/ORGANIZATIONS:")
            for company, count in insights['mentioned_companies']:
                print(f"• {company} ({count} mentions)")
        
        print("\n📰 TOP ARTICLES:")
        for i, article in enumerate(insights['articles'], 1):
            print(f"{i}. {article['title']}")
            print(f"   Source: {article['source']} | Published: {article['published_at']}")
            print(f"   URL: {article['url']}")
            print()
        
        print("=" * 80)

def main():
    app = MarketInsightsApp()
    
    print("🔍 Market Insights Generator")
    print("==========================")
    print("Enter a market topic, company name, or industry to analyze")
    
    while True:
        query = input("\nEnter your query (or 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Thank you for using Market Insights Generator. Goodbye!")
            break
        
        print(f"\nAnalyzing market insights for '{query}'...")
        insights = app.get_market_insights(query)
        app.print_insights(insights)

if __name__ == "__main__":
    main()

🔍 Market Insights Generator
Enter a market topic, company name, or industry to analyze



Enter your query (or 'exit' to quit):  bitcoin



Analyzing market insights for 'bitcoin'...

MARKET INSIGHTS: BITCOIN

📊 OVERVIEW:
• Found 25 articles
• Overall market sentiment: POSITIVE
• Sentiment breakdown: 52% positive, 40% neutral, 8% negative

📝 MARKET SUMMARY:
 If you click 'Accept all', we and our partners, including 239 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use …   If you click 'Accept all', we and our partners, including 241 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use …   If you click 'Accept all', we and our partners, including 239 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use …   If you click 'Accept all', we and our partners, including 239 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on 


Enter your query (or 'exit' to quit):  cred



Analyzing market insights for 'cred'...

MARKET INSIGHTS: CRED

📊 OVERVIEW:
• Found 25 articles
• Overall market sentiment: POSITIVE
• Sentiment breakdown: 60% positive, 32% neutral, 8% negative

📝 MARKET SUMMARY:
This is a full disclosure (albeit without actual PoC code), so assuming matching usermode ex…  What devs make of Switch 2.In case you somehow missed it, Nintendo has officially lifted the lid on Switch 2, cueing all sorts of excitement and apprehension from those of us who have been loyally playing our Switch 1s for the past eight years.It's an excitin… Image: Damien McFerran / Nintendo Life
In case you somehow missed it, Nintendo has officially lifted the lid on Switch 2, cueing all sorts of excitement and apprehension from those of us who have be…  The National Payments Corporation of India (NPCI) is preparing a major overhaul of the Unified Payment Interface (UPI) online checkout experience, sparking concerns among smaller payment apps. Seriously though this concept sound

...


Enter your query (or 'exit' to quit):  memecoins



Analyzing market insights for 'memecoins'...

MARKET INSIGHTS: MEMECOINS

📊 OVERVIEW:
• Found 25 articles
• Overall market sentiment: NEUTRAL
• Sentiment breakdown: 28% positive, 56% neutral, 16% negative

📝 MARKET SUMMARY:
Deputy director of … A Russian finance ministry official has reportedly said the country should be developing its own stablecoin after a recent freeze on wallets linked to the sanctioned Russian exchange Garantex by US a…  Ethena Labs, the developer of the USDe synthetic dollar (USDe), and financial technology company Securitize, released a preliminary roadmap for their upcoming Converge network, a high-throughput blockchain focused on real-world assets and decentralized financ… Ethena Labs, the developer of the USDe synthetic dollar (USDe), and financial technology company Securitize, released a preliminary roadmap for their upcoming Converge network, a high-throughput bloc…  Crypto exchange Coinbase has distanced its blockchain network Base from a memecoin it sha


Enter your query (or 'exit' to quit):  pump fun



Analyzing market insights for 'pump fun'...

MARKET INSIGHTS: PUMP FUN

📊 OVERVIEW:
• Found 24 articles
• Overall market sentiment: POSITIVE
• Sentiment breakdown: 38% positive, 54% neutral, 8% negative

📝 MARKET SUMMARY:
Transcripts are generated using a combination of automated software and human trans…  New Report Explores the Viral TikTok Blue Salt Trick for Men, Uncovers the Real Science Behind Nitric Oxide, and Reveals Why Nitric Boost Is Emerging as 2025’s Top Natural Supplement for Erectile Dysfunction Support New Report Explores the Viral TikTok Blue S… New York City, April 19, 2025 (GLOBE NEWSWIRE) -- 
Understanding the Male Performance Crisis in 2025
Why So Many Men Are Struggling Right Now
Across the world, millions of men are silently battlin…  Masyn Winn's pending return to St. Louis Cardinals limits the potential playing time for Thomas Saggese. Along with adding cross-pl…  Featuring supported day rides, nightly entertainment, catered meals, camping, fun educational opp

KeyboardInterrupt: Interrupted by user